In [ ]:
%load_ext autoreload
%autoreload 2
from IPython.core.display import HTML
HTML('<style>.container { width:95% !important; }</style>')
HTML('<style>.output_png {display: table-cell;text-align: center;vertical-align: middle;}</style>')

In [ ]:
%matplotlib inline
import numpy as np
import os
import sys
import matplotlib.pyplot as plt

In [ ]:
import time
timestamp = time.strftime('%Y_%m_%d_%H_%M_%S')
print timestamp

In [ ]:
root_dir = './'

In [ ]:
src_dir = os.path.join(root_dir, 'code')
# add the 'code' directory as one where we can import modules
# prepend it to the path so that it can mask mosubdules with the same name, i.e. eden
sys.path = [src_dir] + sys.path

In [ ]:
out_data_dir = os.path.join(root_dir, timestamp)
if not os.path.exists(out_data_dir): os.mkdir(out_data_dir)

In [ ]:
# configure logger
from eden.util import configure_logging
import logging
logger = logging.getLogger()
log_fname = os.path.join(out_data_dir, 'log.txt')
configure_logging(logger, verbosity=1, filename=log_fname)

In [ ]:
from toolz import curry, compose, pipe, take

In [ ]:
import requests

def save(text, full_out_file_name):
    with open(full_out_file_name, 'w') as f:
        for line in text:
            f.write("%s\n" % line.encode('utf8').strip())

def rfam_uri(family_id, out_data_dir):
    # retrieve the seed sequences from RFAM and save them to file
    uri = 'http://rfam.xfam.org/family/%s/alignment?acc=%s&format=fastau&download=0'%(family_id,family_id) 
    rfam_dir = os.path.join(out_data_dir, 'RNA')
    fname = family_id + '.fa'
    if not os.path.exists(rfam_dir):
        os.mkdir(rfam_dir)
    full_out_file_name = os.path.join(rfam_dir, fname)
    if not os.path.isfile(full_out_file_name):
        text = requests.get(uri).text.split('\n')
        save(text, full_out_file_name)
    return full_out_file_name

In [ ]:
rfam_id = 'RF00005' #tRNA
data = rfam_uri(rfam_id, out_data_dir)

from eden.converter.fasta import fasta_to_sequence
seqs = list(fasta_to_sequence(data))

In [ ]:
header, seq = seqs[3]
print header
print seq

In [ ]:
# setup
complexity=3
nbits=15

from RaSE import make_fold
fold = make_fold(window_size=70,
                  max_bp_span=70,
                  avg_bp_prob_cutoff=0.1,
                  hard_threshold=0.5,
                  max_num_edges=2,
                  no_lonely_bps=True,
                  nesting=True)

In [ ]:
from RaSE import make_fold_vectorize, stability
fold_vectorize = make_fold_vectorize(complexity=complexity,nbits=nbits,fold=fold)
snips, scores = stability(seq, alphabet='ACGU',fold_vectorize=fold_vectorize)

print snips
print scores

In [ ]:
from RaSE import rna_structural_stability_estimate
rase = curry(rna_structural_stability_estimate)(k=5, complexity=complexity, nbits=nbits, fold=fold)
for line in rase(seq): print(line)

In [ ]:
from RaSE import draw
draw(seq, fold=fold, complexity=complexity, nbits=nbits)

---